## Measuring shift on AuxTel Hysteresis run - 25-Jan-23

Craig Lage

In [ ]:
import sys, time, os, asyncio, glob

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
from astropy.table import Table
from astroquery.astrometry_net import AstrometryNet
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.bestEffort import BestEffortIsr
from lsst.summit.utils.plotting import plot
from lsst.summit.utils import quickSmooth
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

In [ ]:
butler = butlerUtils.makeDefaultLatissButler(embargo=True)

In [ ]:
dataId = dict(day_obs=20240528, detector=0, seq_num=335)
bestEffort = BestEffortIsr(embargo=True)
bestEffort.getExposure(dataId)
dataType = 'quickLookExp'
exp = butler.get(dataType, dataId)

In [ ]:
%matplotlib inline
plot(exp)

In [ ]:
# Run one just to get it working
expId = 2022121300200
rawExp = butler.get('raw', detector=0, exposure=expId)
biasExp = butler.get('bias', detector=0, exposure=expId)
defectExp = butler.get('defects', detector=0, exposure=expId)
isrResult = isrTask.run(rawExp, bias=biasExp, defects=defectExp)
exp = isrResult.exposure

In [ ]:
charConfig = CharacterizeImageConfig()
charConfig.doMeasurePsf = False
charConfig.doApCorr = False
charConfig.doDeblend = False
charConfig.repair.doCosmicRay = False
charConfig.repair.doInterpolate = True   
charConfig.detection.minPixels = 500
charTask = CharacterizeImageTask(config=charConfig)

mData = exp.getMetadata()
charResult = charTask.run(exp)
sourceCatalog = charResult.sourceCat
maxFlux = np.nanmax(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'])
selectBrightestSource = sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.99
brightestSource = sourceCatalog.subset(selectBrightestSource)
brightestCentroid = (brightestSource['base_SdssCentroid_x'][0], \
                     brightestSource['base_SdssCentroid_y'][0])
brightCatalog = sourceCatalog.subset(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.001)
print(f"exposure:{exposure}. Found {len(sourceCatalog)} sources, {len(brightCatalog)} bright sources")
print(f"Brightest centroid at {brightestCentroid}")
pixelScale = exp.getWcs().getPixelScale().asArcseconds()
print(f"Pixel scale = {pixelScale} arcseconds/pixel")
detectorCenter = (2048.5, 2000.5)
xoff = (brightestCentroid[0] - detectorCenter[0]) * pixelScale
yoff = (brightestCentroid[1] - detectorCenter[1]) * pixelScale
print(f"xoff = {xoff}, yoff = {yoff} arcseconds")
off = np.sqrt(xoff*xoff + yoff*yoff)
print(f"Offset = {off} arcseconds")
rotpa = Angle(mData['ROTPA']*u.deg)
el = Angle(mData['ELSTART'] * u.deg)
az = Angle(mData['AZSTART'] * u.deg)
dec = Angle(mData['DECSTART'] * u.deg)
# Now calculate and plot the Az, El arrows from my algorithm
sinTheta =  np.cos(AUXTEL_LOCATION.lat) / np.cos(dec) * np.sin(az)
cosTheta = (np.sin(el) * np.sin(dec) - np.sin(AUXTEL_LOCATION.lat)) / (np.cos(el) * np.cos(dec))

theta = Angle(np.arcsin(sinTheta))
if cosTheta > 0:
    rotAzEl = rotpa - theta
else:    
    rotAzEl = rotpa - (Angle(180.0 * u.deg) - theta)
print(f"rotAzEl = {rotAzEl.deg} degrees")
offset = (xoff * np.cos(rotAzEl) + yoff * np.sin(rotAzEl), -xoff * np.sin(rotAzEl) + yoff * np.cos(rotAzEl))
print(f"Az_offset = {offset[0]} arcseconds, El_offset = {offset[1]} arcseconds")

In [ ]:
cat = brightCatalog
from matplotlib.colors import LogNorm
%matplotlib inline
Ncenter = (700, 900) # Center of N,E arrows
Nlength = 500.0 # Length of arrows
NcenterAzEl = (3200, 700) # Center of Az/El arrows
Nlabel = 650.0 # Displacement of label from the end of the arrows

plt.figure(figsize=(16,16))
plt.title(f"AuxTel Azimuth hysteresis, {expId}", fontsize = 24)
arr = exp.image.array
arr = np.clip(arr, 1, 100000) # This image has some negative values, and this removes them
img = plt.imshow(arr, norm=LogNorm(vmin=1, vmax=100),  interpolation='Nearest', cmap='gray')

plt.scatter(brightestCentroid[0],brightestCentroid[1]\
            ,color='red', marker='x', s=1000)
# First plot the N,E arrows
rotpa = Angle(mData['ROTPA']*u.deg)
plt.arrow(Ncenter[0],Ncenter[1], -Nlength*np.sin(rotpa), Nlength*np.cos(rotpa),\
    color='yellow', width = 20)
plt.text(Ncenter[0]-Nlabel*np.sin(rotpa),Ncenter[1]+Nlabel*np.cos(rotpa), 'N', \
    color='yellow', fontsize=12, weight='bold')
plt.arrow(Ncenter[0],Ncenter[1], Nlength*np.cos(rotpa), Nlength*np.sin(rotpa),\
    color='yellow', width = 20)
plt.text(Ncenter[0]+Nlabel*np.cos(rotpa),Ncenter[1]+Nlabel*np.sin(rotpa), 'E', \
    color='yellow', fontsize=12, weight='bold')

el = Angle(mData['ELSTART'] * u.deg)
az = Angle(mData['AZSTART'] * u.deg)
dec = Angle(mData['DECSTART'] * u.deg)
# Now calculate and plot the Az, El arrows from my algorithm
sinTheta =  np.cos(AUXTEL_LOCATION.lat) / np.cos(dec) * np.sin(az)
cosTheta = (np.sin(el) * np.sin(dec) - np.sin(AUXTEL_LOCATION.lat)) / (np.cos(el) * np.cos(dec))

theta = Angle(np.arcsin(sinTheta))
if cosTheta > 0:
    rotAzEl = rotpa - theta
else:    
    rotAzEl = rotpa - (Angle(180.0 * u.deg) - theta)
print(theta.deg, rotpa.deg, rotAzEl.deg)
plt.arrow(NcenterAzEl[0],NcenterAzEl[1], -Nlength*np.sin(rotAzEl), Nlength*np.cos(rotAzEl),\
    color='cyan', width = 20)
plt.text(NcenterAzEl[0]-Nlabel*np.sin(rotAzEl),NcenterAzEl[1]+Nlabel*np.cos(rotAzEl), 'EL', \
    color='cyan', fontsize=12, weight='bold')
plt.arrow(NcenterAzEl[0],NcenterAzEl[1], Nlength*np.cos(rotAzEl), Nlength*np.sin(rotAzEl),\
    color='cyan', width = 20)
plt.text(NcenterAzEl[0]+Nlabel*np.cos(rotAzEl),NcenterAzEl[1]+Nlabel*np.sin(rotAzEl), 'AZ', \
    color='cyan', fontsize=12, weight='bold')

plt.text(brightestCentroid[0]-200.0,brightestCentroid[1] - 200.0,  \
    f"Az_offset = {offset[0]:.2f} arcseconds, El_offset = {offset[1]:.2f} arcseconds", \
         color='yellow', fontsize=12, weight='bold')
plt.ylim(0,4000)
plt.savefig(f"/home/c/cslage/u/AuxTel/hysteresis_data/AuxTel_Azimuth_Hysteresis_{expId}_25Jan23.pdf")

In [ ]:
# Now run them all


outfile = "/home/c/cslage/u/AuxTel/hysteresis_data/AuxTel_Azimuth_Hysteresis_25Jan23.txt"
file = open(outfile, "w")
file.write("Exposure\tAz(degrees)\tEl(degrees)\tDeltaAz(sec)\tDeltaEl(sec)\n")
for expId in range(2022121300011, 2022121300322):
    rawExp = butler.get('raw', detector=0, exposure=expId)
    biasExp = butler.get('bias', detector=0, exposure=expId)
    defectExp = butler.get('defects', detector=0, exposure=expId)
    isrResult = isrTask.run(rawExp, bias=biasExp, defects=defectExp)
    exp = isrResult.exposure
    mData = exp.getMetadata()
    rotpa = Angle(mData['ROTPA']*u.deg)
    el = Angle(mData['ELSTART'] * u.deg)
    az = Angle(mData['AZSTART'] * u.deg)
    dec = Angle(mData['DECSTART'] * u.deg)
    # Now calculate and plot the Az, El arrows from my algorithm
    sinTheta =  np.cos(AUXTEL_LOCATION.lat) / np.cos(dec) * np.sin(az)
    cosTheta = (np.sin(el) * np.sin(dec) - np.sin(AUXTEL_LOCATION.lat)) / (np.cos(el) * np.cos(dec))

    theta = Angle(np.arcsin(sinTheta))
    if cosTheta > 0:
        rotAzEl = rotpa - theta
    else:    
        rotAzEl = rotpa - (Angle(180.0 * u.deg) - theta)
    az = mData["AZSTART"]
    el = mData["ELSTART"]

    pixelScale = exp.getWcs().getPixelScale().asArcseconds()
    detectorCenter = (2048.5, 2000.5)

    charResult = charTask.run(exp)
    sourceCatalog = charResult.sourceCat
    maxFlux = np.nanmax(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'])
    selectBrightestSource = sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.99
    brightestSource = sourceCatalog.subset(selectBrightestSource)
    brightestCentroid = (brightestSource['base_SdssCentroid_x'][0], \
                         brightestSource['base_SdssCentroid_y'][0])
    brightCatalog = sourceCatalog.subset(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.001)
    print(f"exposure:{expId}. Found {len(sourceCatalog)} sources, {len(brightCatalog)} bright sources")
    print(f"Brightest centroid at {brightestCentroid}")
    xoff = (brightestCentroid[0] - detectorCenter[0]) * pixelScale
    yoff = (brightestCentroid[1] - detectorCenter[1]) * pixelScale
    off = np.sqrt(xoff*xoff + yoff*yoff)

    offset = (xoff * np.cos(rotAzEl) + yoff * np.sin(rotAzEl), -xoff * np.sin(rotAzEl) + yoff * np.cos(rotAzEl))
    print(f"Az_offset = {offset[0]} arcseconds, El_offset = {offset[1]} arcseconds")
    file.write(f"{expId}\t{az:.4f}\t{el:.4f}\t{offset[0]:.2f}\t{offset[1]:.2f}\n")
file.close()

In [ ]:
# Now read in the data and plot it.
outfile = "/home/c/cslage/u/AuxTel/hysteresis_data/AuxTel_Azimuth_Hysteresis_25Jan23.txt"
data = np.loadtxt(outfile, skiprows=1)
exposures = []
Azs = []
Els = []
deltaAzs = []
deltaEls = []
for j in range(1, data.shape[0]):
    [exposure, Az, El, deltaAz, deltaEl] = data[j]
    Azs.append(Az)
    Els.append(El)
    deltaAzs.append(deltaAz)
    deltaEls.append(deltaEl)
    exposures.append(exposure - 2022121300000)


In [ ]:
%matplotlib inline
plt.figure(figsize=(8, 10))
plt.subplots_adjust(hspace = 0.5)
plt.subplot(2,1,1)
plt.title(f"AuxTel Azimuth Hysteresis DeltaAz", fontsize = 24)
plt.scatter(exposures, deltaAzs)
plt.xlabel("Sequence Number")
plt.ylabel("DeltaAz (arcseconds)")
#plt.xlim(273, 280)
plt.subplot(2,1,2)
plt.title(f"AuxTel Elevation Hysteresis DeltaEl", fontsize = 24)
plt.scatter(exposures, deltaEls)
plt.xlabel("Sequence Number")
plt.ylabel("DeltaEl (arcseconds)")
plt.savefig(f"/home/c/cslage/u/AuxTel/hysteresis_data/AuxTel_Azimuth_Hysteresis_25Jan23.pdf")

In [ ]:
plt.figure(figsize=(8, 10))
plt.subplots_adjust(hspace = 0.5)
plt.subplot(2,1,1)
plt.title(f"AuxTel Azimuth", fontsize = 24)
plt.scatter(exposures, Azs)
plt.xlabel("Sequence Number")
plt.ylabel("Az(degrees)")
plt.subplot(2,1,2)
plt.title(f"AuxTel Elevation ", fontsize = 24)
plt.scatter(exposures, Els)
plt.xlabel("Sequence Number")
plt.ylabel("El(degrees)")
plt.savefig(f"/home/c/cslage/u/AuxTel/hysteresis_data/AuxTel_Azimuth_Hysteresis_Images_25Jan23.pdf")

In [ ]:
# Use this to set the pos and neg exposures.
for i in range(len(exposures)):
    print(i, exposures[i], Azs[i])

In [ ]:
# Set the positive and negative exposures.
# These are for the large excursions
Az_pos = Azs[0:32] + Azs[60:92] + Azs[120:152]
DeltaAz_pos = deltaAzs[0:32] + deltaAzs[60:92] + deltaAzs[120:152]
Az_neg = Azs[34:60] + Azs[92:120] + Azs[154:185]
DeltaAz_neg = deltaAzs[34:60] + deltaAzs[92:120] + deltaAzs[154:185]
exp_pos = exposures[0:32] + exposures[60:92] + exposures[120:152]
exp_neg = exposures[34:60] + exposures[92:120] + exposures[154:185]

In [ ]:
xaxis = np.linspace(160, 200, 100)
fit_pos = np.polyfit(Az_pos, DeltaAz_pos, 1)
fit_neg = np.polyfit(Az_neg, DeltaAz_neg, 1)
yaxis_pos = np.polyval(fit_pos, xaxis)
yaxis_neg = np.polyval(fit_neg, xaxis)

%matplotlib inline
plt.figure(figsize=(8, 8))
plt.subplots_adjust(hspace = 0.5)
plt.subplot(1,1,1)
plt.title(f"AuxTel Azimuth Hysteresis\n Large excursions", fontsize = 18)
plt.plot(xaxis, yaxis_pos, ls='--', lw=4, color='red')
plt.arrow(165, -7.5, 30, 1.2, head_width=0.2, head_length=1.0, color='red')
#plt.text(-25, -1.0, f"Slope={fit_pos[0]:.2f} arcsec/deg, Intercept={fit_pos[1]:.2f} arcsec", color='red')
plt.scatter(Az_neg, DeltaAz_neg, color='blue', marker = 'x', s=100, label = "Negative")
plt.scatter(Az_pos, DeltaAz_pos, color='red', label = "Positive")

plt.plot(xaxis, yaxis_neg, ls='--', lw=4, color='blue')
plt.arrow(195.0, -2.5, -30, -1.0, head_width=0.2, head_length=1.0, color='blue')
#plt.text(-40, -5.0, f"Slope={fit_neg[0]:.2f} arcsec/deg, Intercept={fit_neg[1]:.2f} arcsec", color='blue')
plt.xlabel("Az(degrees)")
plt.ylabel("DeltaAz(arcseconds)")
plt.legend()
plt.savefig(f"/home/c/cslage/u/AuxTel/hysteresis_data/AuxTel_Azimuth_Hysteresis_Summary_Large_25Jan23.pdf")

In [ ]:
#Now bin them and take the difference
Az_neg = np.array(Az_neg)
Az_pos = np.array(Az_pos)
DeltaAz_neg = np.array(DeltaAz_neg)
DeltaAz_pos = np.array(DeltaAz_pos)
lowerEdge = np.linspace(160, 200, 9)
diffs = []
azs = []
for lower in lowerEdge:
    upper = lower + 5.0
    azs.append((upper + lower) / 2.0)
    meanNeg = np.mean(DeltaAz_neg[np.where((Az_neg>lower) & (Az_neg<upper))])
    meanPos = np.mean(DeltaAz_pos[np.where((Az_pos>lower) & (Az_pos<upper))])
    diffs.append(meanPos - meanNeg)

plt.figure(figsize=(8, 8))
plt.subplots_adjust(hspace = 0.5)
plt.subplot(1,1,1)
plt.title(f"AuxTel Azimuth Hysteresis \n averaged in 5 degree azimuth bins", fontsize = 18)
plt.plot(azs, diffs, marker = 'x')
plt.ylabel("Positive going mean - Negative going mean (arcseconds)") 
plt.xlabel("Azimuth (degrees)")
plt.savefig(f"/home/c/cslage/u/AuxTel/hysteresis_data/AuxTel_Azimuth_Hysteresis_Binned_Diffs_Large_25Jan23.pdf")

In [ ]:
# Set the positive and negative exposures.
# These are for the small excursions
Az_pos = Azs[182:206] + Azs[224:248] + Azs[268:290]
DeltaAz_pos = deltaAzs[182:206] + deltaAzs[224:248] + deltaAzs[268:290]
Az_neg = Azs[205:225] + Azs[253:267] + Azs[292:309]
DeltaAz_neg = deltaAzs[205:225] + deltaAzs[253:267] + deltaAzs[292:309]
exp_pos = exposures[182:206] + exposures[224:248] + exposures[268:290]
exp_neg = exposures[205:225] + exposures[253:267] + exposures[292:309]

In [ ]:
xaxis = np.linspace(150, 220, 100)
%matplotlib inline
plt.figure(figsize=(8, 8))
plt.subplots_adjust(hspace = 0.5)
plt.subplot(1,1,1)
plt.title(f"AuxTel Azimuth Hysteresis\n Small excursions", fontsize = 18)
#plt.plot(xaxis, yaxis_pos, ls='--', color='red')
#plt.text(-25, -1.0, f"Slope={fit_pos[0]:.2f} arcsec/deg, Intercept={fit_pos[1]:.2f} arcsec", color='red')
plt.scatter(Az_neg, DeltaAz_neg, color='blue', marker = 'x', s=100, label = "Negative")
plt.scatter(Az_pos, DeltaAz_pos, color='red', label = "Positive")

#plt.plot(xaxis, yaxis_neg, ls='--', color='blue')
#plt.text(-40, -5.0, f"Slope={fit_neg[0]:.2f} arcsec/deg, Intercept={fit_neg[1]:.2f} arcsec", color='blue')
plt.xlabel("Az(degrees)")
plt.ylabel("DeltaAz(arcseconds)")
plt.legend()
plt.savefig(f"/home/c/cslage/u/AuxTel/hysteresis_data/AuxTel_Azimuth_Hysteresis_Summary_Small_25Jan23.pdf")

In [ ]:
#Now bin them and take the difference
Az_neg = np.array(Az_neg)
Az_pos = np.array(Az_pos)
DeltaAz_neg = np.array(DeltaAz_neg)
DeltaAz_pos = np.array(DeltaAz_pos)
lowerEdge = np.linspace(176, 186, 11)
diffs = []
azs = []
for lower in lowerEdge:
    upper = lower + 1.0
    azs.append((upper + lower) / 2.0)
    meanNeg = np.mean(DeltaAz_neg[np.where((Az_neg>lower) & (Az_neg<upper))])
    meanPos = np.mean(DeltaAz_pos[np.where((Az_pos>lower) & (Az_pos<upper))])
    diffs.append(meanPos - meanNeg)

plt.figure(figsize=(8, 8))
plt.subplots_adjust(hspace = 0.5)
plt.subplot(1,1,1)
plt.title(f"AuxTel Azimuth Hysteresis \n averaged in 1 degree azimuth bins", fontsize = 18)
plt.plot(azs, diffs, marker = 'x')
plt.ylabel("Positive going mean - Negative going mean (arcseconds)") 
plt.xlabel("Azimuth (degrees)")
plt.savefig(f"/home/c/cslage/u/AuxTel/hysteresis_data/AuxTel_Azimuth_Hysteresis_Binned_Diffs_Small_25Jan23.pdf")